### Question01:

In [109]:
from Bio import SeqIO
import argparse

# Create CLI for user inputting
parser = argparse.ArgumentParser(description='Assembly a DNA string based on sequences in a FASTA file.')
parser.add_argument('infile', help='Input FASTA file name')
args = parser.parse_args()

# Read sequence from input file
record = SeqIO.read(args.infile, "fasta")
sequence = record.seq

def count_kmers(sequence, k_size):
    '''
    Return a dictionary with key is kmer and value is count
    '''
    data = {}
    size = len(sequence)
    for i in range(size - k_size + 1):
        kmer = sequence[i: i + k_size]
        try:
            data[kmer] += 1
        except KeyError:
            data[kmer] = 1
    # Remove kmers with count 1 from the data dictionary
    data = {kmer: count for kmer, count in data.items() if count != 1}
    return data

# Separate key and value from the data dictionary
kmers = False
for i in range(10, len(sequence)):
    kmer_counts = count_kmers(sequence, i)
    if kmer_counts:
        for kmer, count in kmer_counts.items():
            print(f"{kmer}:{count}")
        kmers = True

GTTGGAAACG:2
TTGGAAACGA:2
TGGAAACGAC:2
GGAAACGACT:2
GAAACGACTG:2
AAACGACTGC:2
AACGACTGCT:2
ATCTTCTTGT:3
TCTTCTTGTT:3
CTTCTTGTTT:3
TTCTTGTTTT:2
TCTTGTTTTT:2
CTTGTTTTTA:2
TTGTTTTTAA:2
TGTTTTTAAA:2
GTTTTTAAAA:2
CCACATTGGG:2
CACATTGGGA:3
ACATTGGGAC:3
CATTGGGACT:3
ATTGGGACTG:3
TTATGGTCTA:2
GTTGGAAACGA:2
TTGGAAACGAC:2
TGGAAACGACT:2
GGAAACGACTG:2
GAAACGACTGC:2
AAACGACTGCT:2
ATCTTCTTGTT:3
TCTTCTTGTTT:3
CTTCTTGTTTT:2
TTCTTGTTTTT:2
TCTTGTTTTTA:2
CTTGTTTTTAA:2
TTGTTTTTAAA:2
TGTTTTTAAAA:2
CCACATTGGGA:2
CACATTGGGAC:3
ACATTGGGACT:3
CATTGGGACTG:3
GTTGGAAACGAC:2
TTGGAAACGACT:2
TGGAAACGACTG:2
GGAAACGACTGC:2
GAAACGACTGCT:2
ATCTTCTTGTTT:3
TCTTCTTGTTTT:2
CTTCTTGTTTTT:2
TTCTTGTTTTTA:2
TCTTGTTTTTAA:2
CTTGTTTTTAAA:2
TTGTTTTTAAAA:2
CCACATTGGGAC:2
CACATTGGGACT:3
ACATTGGGACTG:3
GTTGGAAACGACT:2
TTGGAAACGACTG:2
TGGAAACGACTGC:2
GGAAACGACTGCT:2
ATCTTCTTGTTTT:2
TCTTCTTGTTTTT:2
CTTCTTGTTTTTA:2
TTCTTGTTTTTAA:2
TCTTGTTTTTAAA:2
CTTGTTTTTAAAA:2
CCACATTGGGACT:2
CACATTGGGACTG:3
GTTGGAAACGACTG:2
TTGGAAACGACTGC:2
TGGAAACGACT

### Question02:

In [112]:
import numpy as np
from Bio import SeqIO
import logging

# Create log file
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
f = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh = logging.FileHandler('Question2.log')
fh.setFormatter(f)
logger.addHandler(fh)

logger.debug('Start of the program.')

# Read sequences from Seq02.fasta
sequences = []
with open("Seq02.fasta", "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        sequences.append(str(record.seq))

# Check if all records have the same length
sequence_length = len(sequences[0])
all_same_length = all(len(seq) == sequence_length for seq in sequences)
if all_same_length:
    logger.info('All sequences have the same length.')
else:
    logger.debug("Sequences have different lengths.")

# Determine the consensus sequence
consensus = np.empty(sequence_length, dtype='U1')
for pos in range(sequence_length):
    column = np.array([seq[pos] for seq in sequences])
    unique, counts = np.unique(column, return_counts=True)
    max_count = np.max(counts)
    bases_with_max_count = unique[counts == max_count]

    if len(bases_with_max_count) == 1:
        consensus[pos] = bases_with_max_count[0]
    else:
        consensus[pos] = 'N'

# Convert the consensus array to a string
consensus_sequence = ''.join(consensus)

# Print the consensus sequence
print(consensus_sequence)
logger.info(consensus_sequence)
logger.debug('End of the program.')

TTACGCTGGCGGCACGCCTAATACATGCAAGTCGAACGGAAGTATAAGCAGCAATTACACTTTAGTGGCA


### Question03:

In [111]:
import argparse
from Bio import SeqIO
import logging

# Create CLI for user inputting
parser = argparse.ArgumentParser(description='Assembly a DNA string based on sequences in a FASTA file.')
parser.add_argument('infile', help='Input FASTA file name')
args = parser.parse_args()

# Create log file
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
f = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh = logging.FileHandler('Question3.log')
fh.setFormatter(f)
logger.addHandler(fh)

logger.debug('Start of the program.')

# Read sequences from Seq04.fasta
sequences = []
with open(args.infile, "r") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(record)

# Specify the minimum overlap length
min_overlap = 3

def find_overlap_endseq1_startseq2(seq1, seq2, min_overlap):
    '''
    Return the length of maximum overlap by compare end of sequence1 with start of sequence2
    '''
    max_overlap = 0
    for pos in range(len(seq2) - min_overlap + 1):
        if seq1.endswith(seq2[:len(seq2) - pos]):
            overlap_length = len(seq2) - pos
            if overlap_length > max_overlap:
                max_overlap = overlap_length

    return max_overlap

def find_overlap_endseq2_startseq1(seq1, seq2, min_overlap):
    '''
    Return the length of maximum overlap by compare end of sequence2 with start of sequence1
    '''
    max_overlap = 0
    for pos in range(len(seq1) - min_overlap + 1):
        if seq2.endswith(seq1[:len(seq1) - pos]):
            overlap_length = len(seq1) - pos
            if overlap_length > max_overlap:
                max_overlap = overlap_length
          
    return max_overlap

def assemble_sequences(sequences):
    '''
    Assemble two sequences
    '''
    assembled_sequence = str(sequences[0].seq)
    current_sequence = str(sequences[1].seq)
    
    for pos in range(1, len(sequences)):
        
        current_sequence = str(sequences[pos].seq)
        overlap_endseq1_startseq2 = find_overlap_endseq1_startseq2(assembled_sequence,current_sequence, min_overlap)
        overlap_endseq2_startseq1 = find_overlap_endseq2_startseq1(assembled_sequence,current_sequence ,min_overlap)
        logger.info('endseq1_position: ' + str(overlap_endseq1_startseq2))
        logger.info('endseq2_position: ' + str(overlap_endseq2_startseq1))
        if overlap_endseq2_startseq1 == overlap_endseq1_startseq2 == 0:
            if len(assembled_sequence) > len(current_sequence):
                assembled_sequence == assembled_sequence
            else:
                assembled_sequence == current_sequence    
        elif overlap_endseq1_startseq2 > overlap_endseq2_startseq1:
            assembled_sequence += current_sequence[overlap_endseq1_startseq2:]    
        else:
            assembled_sequence = current_sequence[:-overlap_endseq2_startseq1] + assembled_sequence
            
    return assembled_sequence

# Assemble sequences with overlaps
assembled_dna_string = assemble_sequences(sequences)

# Print the assembled DNA string
print(assembled_dna_string)
logger.info(assembled_dna_string)
logger.debug('End of the program.')


GCTCAACATTGTGATCTTCTTGTTTTTAAAATGCTATAAAAACTGTTTAGCTAGAGTAAGATAGAGGCAAGT
